<a href="https://colab.research.google.com/github/vatshivam/CU-course-similarity/blob/main/course_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install transformers
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=ebbb522c9242418aeb93c7991c12ed80fc425ed851b86eadf7fbe0d74e27cda4
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Active Courses.xlsx to Active Courses.xlsx


In [ ]:
import io
xlsx_file = io.BytesIO(uploaded.get('Active Courses.xlsx'))

In [ ]:
df = pd.read_excel(xlsx_file)

In [ ]:
df.head()

,EFFDT,EFF STATUS,SBJCT CD,CATALOG NUM,CRSE ID,CRSE LD,CRSE TTL LD,CRSE CATALOG LD,GRADE BASIS CD,SSR COMP CD,CATALOG PRINT,ACAD CAR CD,ACAD GRP CD,ACAD ORG CD,INSTRCTN MODE LD,MIN UNITS,MAX UNITS
0,2016-05-01,A,XFCR,2999SA,100001,Lower Division General Credit,Lower Division General Credit,-,OPT,LEC,N,UGRD,CRSS,B-CRSS,In Person,0.1,99.0
1,2016-05-01,A,XFCR,2999TC,100001,Lower Division General Credit,Lower Division General Credit,-,OPT,LEC,N,UGRD,CRSS,B-CRSS,In Person,0.1,99.0
2,2016-05-01,A,XFCR,3999SA,100002,Upper Division General Credit,Upper Division General Credit,-,OPT,LEC,N,UGRD,CRSS,B-CRSS,In Person,3.0,3.0
3,2016-05-01,A,XFCR,3999TC,100002,Upper Division General Credit,Upper Division General Credit,-,OPT,LEC,N,UGRD,CRSS,B-CRSS,In Person,3.0,3.0
4,2016-05-01,A,XFCR,4999SA,100003,Upper Division General Credit,Upper Division General Credit,-,OPT,LEC,N,UGRD,CRSS,B-CRSS,In Person,0.1,99.0


In [ ]:
# removing rows with no course catalog information
df = df[df['CRSE CATALOG LD']!='-']

In [ ]:
df.head()

,EFFDT,EFF STATUS,SBJCT CD,CATALOG NUM,CRSE ID,CRSE LD,CRSE TTL LD,CRSE CATALOG LD,GRADE BASIS CD,SSR COMP CD,CATALOG PRINT,ACAD CAR CD,ACAD GRP CD,ACAD ORG CD,INSTRCTN MODE LD,MIN UNITS,MAX UNITS
12,2018-08-02,A,ACCT,3220,100190,Corporate Financial Rprtng 1,Corporate Financial Reporting 1,First of a two-course sequence intended to pro...,OPT,LEC,Y,UGRD,BUSN,B-ACCT,In Person,3.0,3.0
13,2017-08-01,A,ACCT,3230,100191,Corporate Financial Rprtng 2,Corporate Financial Reporting 2,Second in a two-course sequence building and e...,OPT,LEC,Y,UGRD,BUSN,B-ACCT,In Person,3.0,3.0
14,2018-08-02,A,ACCT,3320,100192,Cost Management,Cost Management,Provides cost analysis for the support of mana...,OPT,LEC,Y,UGRD,BUSN,B-ACCT,In Person,3.0,3.0
17,2017-08-01,A,ACCT,4240,100194,Advanced Financial Accounting,Advanced Financial Accounting,Examines advanced financial accounting theory ...,OPT,LEC,Y,UGRD,BUSN,B-ACCT,In Person,3.0,3.0
18,2019-08-01,A,ACCT,4250,100195,Financial Statement Analysis,Financial Statement Analysis,Focuses on the use of U.S. and international a...,OPT,LEC,Y,UGRD,BUSN,B-ACCT,In Person,3.0,3.0


In [ ]:
s1 = df['CRSE CATALOG LD'].iloc[100]
s1

'Archaeological evidence for Native American ways of life on the North American Great Plains from the initial peopling of the region into the 19th century. Recommended prerequisite: ANTH 2200. Same as ANTH 5270.'

In [ ]:
s2=df['CRSE CATALOG LD'].iloc[1000]
s2

'Covers prehistoric Aegean through the fourth century B.C.E., considering architecture, pottery, painting, sculpture and personal ornament. Societal customs such as use of space and burial patterns are considered as well as art and its uses, to help understand developments in Greek culture. Same as ARTH 2039. Formerly CLAS 3039.'

In [ ]:
# encode sentences to get their embeddings
model = SentenceTransformer('stsb-roberta-large')
embedding1 = model.encode(s1, convert_to_tensor=True)
embedding2 = model.encode(s2, convert_to_tensor=True)

In [ ]:
# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)

In [ ]:
print("Similarity score:", cosine_scores.item())

Similarity score: 0.47695615887641907


In [ ]:
df['SBJCT CD'].value_counts()

In [ ]:
for index,r1 in df[df['SBJCT CD']=='MSBX'].iterrows():
  for index,r2 in df[df['SBJCT CD']=='DTSC'].iterrows():
    embedding1 = model.encode(r1['CRSE CATALOG LD'], convert_to_tensor=True)
    embedding2 = model.encode(r2['CRSE CATALOG LD'], convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
    print("Similarity scores between:",r1['SBJCT CD'],r1['CATALOG NUM'],r2['SBJCT CD'],r2['CATALOG NUM'],cosine_scores.item())

Similarity scores between: MSBX 5415 DTSC 5301 0.637967586517334
Similarity scores between: MSBX 5415 DTSC 5302 0.6350412368774414
Similarity scores between: MSBX 5415 DTSC 5303 0.447349488735199
Similarity scores between: MSBX 5415 DTSC 5001 0.6334174871444702
Similarity scores between: MSBX 5415 DTSC 5002 0.5470665693283081
Similarity scores between: MSBX 5415 DTSC 5003 0.6729437112808228
Similarity scores between: MSBX 5415 DTSC 5020 0.4710911512374878
Similarity scores between: MSBX 5415 DTSC 5501 0.5113711953163147
Similarity scores between: MSBX 5415 DTSC 5810 0.6494451761245728
Similarity scores between: MSBX 5415 DTSC 5840 0.3587146997451782
Similarity scores between: MSBX 5415 DTSC 5900 0.23415425419807434
Similarity scores between: MSBX 5415 DTSC 5930 0.5745105743408203
Similarity scores between: MSBX 5410 DTSC 5301 0.5158755779266357
Similarity scores between: MSBX 5410 DTSC 5302 0.47052788734436035
Similarity scores between: MSBX 5410 DTSC 5303 0.44811171293258667
Similarit

MSBX
MSBX
MSBX
MSBX
MSBX
MSBX
MSBX
MSBX
MSBX
MSBX
MSBX
MSBX
MSBX
MSBX
MSBX
MSBX
MSBX
MSBX


MSBX 5480 DTSC 5303
MSBX 5405 DTSC 5003
MSBX 5405 DTSC 5301
MSBX 5405 DTSC 5302
MSBX 5310 DTSC 5020
MSBX 5430 DTSC 5501 0.678718090057373
MSBX 5420 DTSC 5001 0.6530780792236328
MSBX 5420 DTSC 5301 0.6221216320991516
MSBX 5430 DTSC 5001 0.6984046697616577
MSBX 5430 DTSC 5020 0.8081475496292114

In [ ]:
print(df[(df['CATALOG NUM']=='5430') & (df['SBJCT CD']=='MSBX')]['CRSE CATALOG LD'].iloc[0])

Introduces advanced multivariate regression analysis and residual diagnostics, logistic regression, analysis of variance (ANOVA and MANOVA), time series models, and analysis of categorical variables. R, an open source programming language for statistical computing and graphics, will be used. It is assumed students have mastery of introductory statistics topics including descriptive tools, inference, and ordinary least squares regression.


In [ ]:
print(df[(df['CATALOG NUM']=='5020') & (df['SBJCT CD']=='DTSC')]['CRSE CATALOG LD'].iloc[0])

Introduction to statistical concepts, models, and algorithms of machine learning. Reviews supervised learning for regression, and introduces classification methods, discriminant analysis, resampling methods, classification and regression trees, random forests and associated tuning, diagnostics, and performance evaluation; also covers unsupervised learning for clustering and principal components analysis. Course uses R as the primary programming language.


Introduces advanced multivariate regression analysis and residual diagnostics, logistic regression, analysis of variance (ANOVA and MANOVA), time series models, and analysis of categorical variables. R, an open source programming language for statistical computing and graphics, will be used. It is assumed students have mastery of introductory statistics topics including descriptive tools, inference, and ordinary least squares regression.




---

Introduction to statistical concepts, models, and algorithms of machine learning. Reviews supervised learning for regression, and introduces classification methods, discriminant analysis, resampling methods, classification and regression trees, random forests and associated tuning, diagnostics, and performance evaluation; also covers unsupervised learning for clustering and principal components analysis. Course uses R as the primary programming language.
